# OCR Project:

In [ ]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 

# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


!pip install --upgrade pip
!pip install opencv-contrib-python

!pip install imutils

     |████████████████████████████████| 1.5MB 9.2MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1


In [ ]:
# imports

import matplotlib
matplotlib.use("Agg")
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import build_montages
import matplotlib.pyplot as plt
import numpy as np
import argparse
import cv2
from tensorflow.keras.datasets import mnist
from keras.applications.resnet import ResNet50
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Loading Datasets:

In [ ]:
# Load Kaggle A-Z dataset

def load_az_dataset (datasetPath):
    data = []
    labels = []
    
    for row in open(datasetPath):
        #print("HI")
        row = row.split(",")
        label = int(row[0])
        image = np.array([int(x) for x in row[1:]], dtype = "uint8")
        
        image = image.reshape((28, 28))
        
        data.append(image)
        labels.append(label)
        
    data = np.array(data, dtype = "float32")
    labels = np.array(labels, dtype = "int")
        
    return (data, labels)

In [ ]:
def load_mnist_dataset ():
    ((trainData, trainLabels), (testData, testLabels)) = mnist.load_data()
    data = np.vstack([trainData, testData])
    labels = np.hstack([trainLabels, testLabels])
    
    return (data, labels)

### Training Model:

In [ ]:
EPOCHS = 300
INIT_LR = 3e-1
BS = 128

print("[INFO] loading datasets...")
(azData, azLabels) = load_az_dataset("/content/drive/MyDrive/Colab Notebooks/Character Recog/A_Z Handwritten Data.csv")
(digitsData, digitsLabels) = load_mnist_dataset()

[INFO] loading datasets...
11493376/11490434 [==============================] - 0s 0us/step


In [ ]:
azLabels += 10

data = np.vstack([azData, digitsData])
labels = np.hstack([azLabels, digitsLabels])

data = [cv2.resize(image, (32, 32)) for image in data]
data = np.array(data, dtype = "float32")

data = np.expand_dims(data, axis = -1)
data /= 255.0

In [ ]:
le = LabelBinarizer()
labels = le.fit_transform(labels)
counts = labels.sum(axis = 0)
classTotals = labels.sum(axis = 0)
classWeight = {}

for i in range(0, len(classTotals)):
    classWeight[i] = classTotals.max() / classTotals[i]
  
(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size = 0.20, random_state = 42)

In [ ]:
aug = ImageDataGenerator(
    rotation_range = 10,
    zoom_range = 0.05,
    width_shift_range = 0.1,
    height_shift_range = 0.1,
    shear_range = 0.15,
    horizontal_flip = False,
    fill_mode = "nearest" )

In [ ]:
trainSize = 100000 #change to increase training set size
testSize = 20000 #change to increase test size

trainX = trainX[:trainSize, ...]
trainY = trainY[:trainSize, ...]

print(trainX.shape)

testX = testX[:testSize, ...]
testY = testY[:testSize, ...]

(100000, 32, 32, 1)


In [ ]:
def ResNet_Model(a, b, c):
    baseModel = ResNet50(weights = "imagenet", include_top = False, input_tensor = Input(shape = (32, 32, 3)))

    headModel = baseModel.output
    print(headModel.shape)
    #headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
    headModel = Flatten(name="flatten")(headModel)
    headModel = Dense(256, activation="relu")(headModel)
    headModel = Dropout(0.5)(headModel)
    headModel = Dense(len(le.classes_), activation="softmax")(headModel)

    model = Model(inputs=baseModel.input, outputs=headModel)
    return model

In [ ]:
print("[INFO] compiling model...")
opt = SGD(lr = INIT_LR, decay = 1*INIT_LR / (3*EPOCHS))
model = ResNet_Model(32, 32, 1)
model.compile(loss = "categorical_crossentropy", optimizer = opt, metrics = ["accuracy"])

[INFO] compiling model...
94773248/94765736 [==============================] - 1s 0us/step
(None, 1, 1, 2048)


In [ ]:
sh = trainX.shape[:]
train_X = np.zeros([sh[0], sh[1], sh[2], 3])
#print(train_X.shape)

# for i in range(0, trainX.shape[0]):
#     train_X[i, ...] = cv2.cvtColor(trainX[i, ...], cv2.COLOR_GRAY2BGR)

train_X[..., :] = trainX

# for i in range(0, sh[0]):
#     for j in range(0, sh[1]):
#         for k in range(0, sh[2]):
#             train_X[i, j, k, :] = trainX[i, j, k, 0]
#train_X = np.repeat(trainX[-1], 3, -1)
print(train_X.shape)
#trainY = np.repeat(trainY[-1], 3, -1)

test_X = np.zeros([testX.shape[0], testX.shape[1], testX.shape[2], 3])

# for i in range(0, testX.shape[0]):
#     test_X[i, ...] = cv2.cvtColor(testX[i, ...], cv2.COLOR_GRAY2BGR)

test_X[..., :] = testX
# for i in range(0, testX.shape[0]):
#     for j in range(0, testX.shape[1]):
#         for k in range(0, testX.shape[2]):
#             test_X[i, j, k, :] = testX[i, j, k, 0]
print(test_X.shape)
#testX = np.repeat(testX[-1], 3, -1)
#testY = np.repeat(testY[-1], 3, -1)

(100000, 32, 32, 3)
(20000, 32, 32, 3)


In [ ]:
# train_X.shape

In [ ]:
# model = tf.keras.models.load_model("../input/trained-model-file/Trained Character Recognizer Model")

In [ ]:
from numpy import argmax
print("[INFO] training network...")
H = model.fit(
    aug.flow(train_X, trainY, batch_size = BS),
    validation_data = (test_X, testY),
    steps_per_epoch = len(trainX) // BS,
    epochs = EPOCHS,
    class_weight = classWeight,
    verbose = 1)

labelNames = "0123456789"
labelNames += "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
labelNames = [l for l in labelNames]

#print("[INFO] evaluating network...")
#predictions = model.predict(test_X, batch_size = BS)
#print(classification_report(testY, argmax(axis = 1), predictions.argmax(axis = 1), target_names = labelNames))

[INFO] training network...
Epoch 1/300
781/781 [==============================] - 116s 133ms/step - loss: 56.1331 - accuracy: 0.0350 - val_loss: 3.4456 - val_accuracy: 0.0861
Epoch 2/300
781/781 [==============================] - 67s 85ms/step - loss: 16.5569 - accuracy: 0.0718 - val_loss: 3.3696 - val_accuracy: 0.1015
Epoch 3/300
781/781 [==============================] - 67s 86ms/step - loss: 16.3289 - accuracy: 0.0872 - val_loss: 3.2618 - val_accuracy: 0.0999
Epoch 4/300
781/781 [==============================] - 67s 86ms/step - loss: 15.9334 - accuracy: 0.0988 - val_loss: 3.0844 - val_accuracy: 0.1237
Epoch 5/300
781/781 [==============================] - 67s 86ms/step - loss: 15.2858 - accuracy: 0.1153 - val_loss: 2.8235 - val_accuracy: 0.1260
Epoch 6/300
781/781 [==============================] - 67s 86ms/step - loss: 14.6215 - accuracy: 0.1484 - val_loss: 2.6338 - val_accuracy: 0.2314
Epoch 7/300
781/781 [==============================] - 68s 87ms/step - loss: 13.8436 - accuracy

In [ ]:
model.save("Trained Character Recognizer Model", save_format = "h5")

In [ ]:
from numpy import argmax
print("[INFO] evaluating network...")
predictions = model.predict(test_X, batch_size = BS)
print(classification_report(testY.argmax(axis = 1), predictions.argmax(axis = 1), target_names = labelNames))

In [ ]:
N = np.arange(0, EPOCHS)
plt.style.use("ggplot")
plt.figure()
plt.plot(N, H.history["loss"], label = "train_loss")
plt.plot(N, H.history["val_loss"], label = 'val_loss')
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc = "lower left")
plt.savefig("PLOT")

In [ ]:
labelNames = "0123456789"
labelNames += "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
labelNames = [l for l in labelNames]

In [ ]:
images = []

for i in np.random.choice(np.arange(0, len(testY)), size = (49,)):
    probs = model.predict(test_X[np.newaxis, i])
    prediction = probs.argmax(axis = 1)
    label = labelNames[prediction[0]]
    
    image = (testX[i] * 255).astype("uint8")
    color = (0, 255, 0)
    
    if prediction[0] != np.argmax(testY[i]):
        color = (0, 0, 255)
    
    image = cv2.merge([image] * 3)
    image = cv2.resize(image, (96, 96), interpolation = cv2.INTER_LINEAR)
    cv2.putText(image, label, (5, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.75, color, 2)
    
    images.append(image)

montage = build_montages(images, (96, 96), (7, 7))[0]

# plt.imshow(montage)
# plt.show()

cv2.imwrite("Montage.png", montage)
cv2.imshow("OCR Results", montage) 
cv2.waitKey()
#cv2.destroyAllWindows()